In [2]:
import numpy as np

https://adventofcode.com/2021/day/9

In [5]:
aoc_day = '2021-12-09'
test_input_file = f"../test_inputs/{aoc_day}-input.txt"
input_file = f"../inputs/{aoc_day}-input.txt"
try_test_input = False


def preprocess_input(input:str) -> list:
    input = [x for x in input.split("\n") if x != ""]
    input = [[int(char) for char in x] for x in input]
    input = np.matrix(input)
    return input 

with open(test_input_file, 'r') as f:
    test_input = preprocess_input(f.read())

with open(input_file, 'r') as f:
    input = preprocess_input(f.read())

if try_test_input:
    input = test_input.copy()

input

matrix([[4, 2, 1, ..., 3, 9, 9],
        [4, 3, 2, ..., 6, 7, 8],
        [5, 7, 3, ..., 5, 6, 7],
        ...,
        [0, 2, 4, ..., 2, 1, 9],
        [1, 2, 3, ..., 3, 2, 6],
        [4, 3, 4, ..., 4, 3, 5]])

In [7]:
def is_low_point(x, neighbors):
  return x < min(neighbors)

x_coord = 0
y_coord = 3


def get_neighbors(x_coord,y_coord, grid_shape):
  neighbors = []
  for i in [x_coord-1,x_coord+1]:
    if i >= 0 and i < grid_shape[0]:
      neighbors += [(i, y_coord)]

  for i in [y_coord-1,y_coord+1]:
    if i >= 0 and i < grid_shape[1]:
      neighbors += [(x_coord, i)]
  return neighbors

def is_low_point(x_coord,y_coord, grid):
  neighbors = get_neighbors(x_coord,y_coord,grid.shape)
  neighbor_values = [input[coords] for coords in neighbors]
  is_low = input[x_coord,y_coord] < min(neighbor_values)
  return is_low

is_low_grid = input == -99999 # Hacky way to get grid of conds
low_points_list = []
for row in range(input.shape[0]):
  for col in range(input.shape[1]):
    neighbors = get_neighbors(row,col,input.shape)
    neighbor_values = [input[coords] for coords in neighbors]
    is_low = input[row,col] < min(neighbor_values)
    is_low_grid[row,col] = is_low
    if is_low:
      low_points_list += [(row,col)]

risk_level = input[is_low_grid].sum() + is_low_grid.sum()
print(is_low_grid)
print(input[is_low_grid])
print(risk_level)
print(low_points_list)

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [ True False False ... False  True False]
 [False False False ... False False False]
 [False False False ... False False False]]
[[0 0 0 0 1 3 1 3 1 0 2 2 2 0 1 1 0 0 2 1 0 2 0 0 0 1 3 0 6 2 2 1 1 0 2 1
  1 0 3 1 0 3 0 1 1 0 0 0 4 1 5 0 0 3 2 0 0 2 0 3 1 0 2 0 1 2 0 0 0 2 2 1
  1 5 2 1 0 2 0 1 1 4 1 2 2 1 0 2 1 1 1 0 0 0 2 0 0 2 1 0 0 1 0 4 0 0 0 0
  1 5 1 4 2 2 0 4 2 2 0 1 0 1 0 0 0 1 1 0 3 3 1 1 3 3 0 1 0 2 1 1 0 0 3 0
  0 0 1 0 3 2 1 0 0 1 1 0 0 0 0 0 2 0 0 1 0 0 3 1 0 0 1 3 0 1 0 0 0 0 1 0
  2 2 1 1 4 0 0 3 1 1 6 5 1 0 1 0 0 0]]
417
[(0, 3), (0, 43), (0, 69), (0, 80), (0, 95), (1, 33), (1, 62), (2, 22), (2, 50), (4, 11), (4, 88), (5, 67), (6, 20), (6, 73), (7, 28), (7, 40), (7, 95), (8, 60), (10, 17), (10, 33), (10, 54), (10, 66), (10, 83), (11, 0), (11, 73), (11, 99), (12, 79), (13, 39), (13, 44), (14, 8), (15, 17), (15, 49), (16, 66), (16, 92), (1

In [8]:
def find_basin(x_coord,y_coord,input):
  basin = []
  basin_end_values = [9]
  row = x_coord + 1
  basin += [(x_coord,y_coord)]
  # Move South
  while row < input.shape[0]:
    if input[row, y_coord] in basin_end_values:
      break
    else:
      basin += [(row, y_coord)]
      row += 1

  # Move North
  row = x_coord - 1
  while row >= 0:
    if input[row, y_coord] in basin_end_values:
      break
    else:
      basin += [(row, y_coord)]
      row -= 1

  # Move West
  col = y_coord - 1
  while col >= 0:
    if input[x_coord, col] in basin_end_values:
      break
    else:
      basin += [(x_coord, col)]
      col -= 1

  # Move East
  col = y_coord + 1
  while col < input.shape[1]:
    if input[x_coord, col] in basin_end_values:
      break
    else:
      basin += [(x_coord, col)]
      col += 1
  return basin

def find_full_basin(x_coord,y_coord,input):
  initial_basin = find_basin(x_coord,y_coord,input)
  full_basin = initial_basin
  for b in initial_basin:
    additional_basin = find_basin(b[0],b[1],input)
    new_points = [x for x in additional_basin if x not in full_basin]
    full_basin += new_points
  return full_basin

x_coord,y_coord = 2,2
full_basin = find_full_basin(x_coord,y_coord,input)

basin_sizes = [len(find_full_basin(x[0],x[1],input)) for x in low_points_list]

basin_sizes.sort()
product = 1

for x in basin_sizes[-3:]:
  product *= x

print("Product of sizes of 3 largest basins")
print(product)  

Product of sizes of 3 largest basins
1148965
